# Entrenamos modelos con nuevo dataset

## 1. Nuevo dataset
Sacado de: https://www.kaggle.com/kazanova/sentiment140#

In [1]:
import re
import csv
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import collections
from nltk.metrics.scores import (precision, recall)
import pickle

In [2]:
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
# Esta funcion elimina links y nombres de usuarios
def remove_linksT(text):
    text = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', text)
    text = re.sub('\@\w+', ' ', text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

In [4]:
# Este objeto es nuestro clasificador
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [5]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [17]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive/NLP

/
Mounted at /content/gdrive
ls: cannot access '/mydrive/NLP': No such file or directory


In [ ]:
# Bajamos base de datos
documents = []
all_words = []
instancias = 20000
LB, UB = instancias, 1600000 - instancias
# Guardaremos verbos, advervios y adjetivos
allowed_word_types = ["J","R","V","N"]
with open('/mydrive/NLP/noemoticon.csv', 'r',encoding='latin-1') as file:
    csv_file = csv.reader(file)
    count=0
    for row in csv_file:
        if count < LB or count >= UB:
          # normalizamos el tweet
          text = remove_linksT(row[5].lower())
          # obtenemos todas las palabras
          words = word_tokenize(text)
          pos = nltk.pos_tag(words)
          # Guardamos el tweet y su clasificacion
          row[0] = 'neg' if row[0] == '0' else 'pos'
          documents.append((text, row[0]))
          # Guardamos las palabras
          for w in pos:
            if w[1][0] in allowed_word_types:
              all_words.append(w[0])
        count += 1 

In [ ]:
file = open('/mydrive/NLP/modelos/documents.pickle', 'wb')
pickle.dump(documents, file)
file.close()

In [ ]:
countp = 0
countn = 0
countne = 0
for i in documents:
    if i[1] == 'pos':
        countp += 1
    if i[1] == 'neg':
        countn += 1
    if i[1] == '2':
        countne +=1
print(f'tenemos {countp} tweets positivos')
print(f'tenemos {countn} tweets negativos')
print(f'tenemos {countne} tweets neutrales')
print(f'tenemos {len(documents)} tweets')

In [ ]:
# vemos que tanto se repite cada palabra
all_words = nltk.FreqDist(all_words)
print(f'Tenemos {len(list(all_words.keys()))} palabras')

In [ ]:
# Tomamos las primeras
word_features = list(all_words.keys())[:5000]

In [ ]:
file = open('/mydrive/NLP/modelos/word_features.pickle', 'wb')
pickle.dump(word_features, file)
file.close()

In [ ]:
# Creamos nuestro featureset (que palabras estan en cada tweet y cual es su categoria)
featuresets = [(find_features(rev), category) for (rev, category) in documents]
random.shuffle(featuresets)

In [ ]:
train = int(instancias*2*0.8)
print(train)

In [ ]:
# dividimos en training y test set
testing_set = featuresets[train:]
training_set = featuresets[:train]

In [ ]:
save_classifier = open("/mydrive/NLP/modelos/training.pickle","wb")
pickle.dump(training_set, save_classifier)
save_classifier.close()

save_classifier = open("/mydrive/NLP/modelos/testing.pickle","wb")
pickle.dump(testing_set, save_classifier)
save_classifier.close()

Clasificadores

In [ ]:
# Naive Bayes
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)

In [7]:
save_classifier = open("/mydrive/NLP/modelos/NaiveBayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

FileNotFoundError: ignored

In [8]:
# MultinomialNB
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

NameError: ignored

In [9]:
save_classifier = open("/mydrive/NLP/modelos/MNB.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

FileNotFoundError: ignored

In [10]:
# BernoulliNB
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

NameError: ignored

In [11]:
save_classifier = open("/mydrive/NLP/modelos/BernoulliNB.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()

FileNotFoundError: ignored

In [12]:
# SGDC
SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
print("accuracy percent:", (nltk.classify.accuracy(SGDC_classifier, testing_set))*100)

NameError: ignored

In [13]:
save_classifier = open("/mydrive/NLP/modelos/SGDC.pickle","wb")
pickle.dump(SGDC_classifier, save_classifier)
save_classifier.close()

FileNotFoundError: ignored

In [14]:
# Linear SVC
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

NameError: ignored

In [15]:
save_classifier = open("/mydrive/NLP/modelos/LinearSVC.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()

FileNotFoundError: ignored

In [16]:
# Clasificador por votos
voted_classifier = VoteClassifier(MNB_classifier, BernoulliNB_classifier,
                                 SGDC_classifier, LinearSVC_classifier)
print(f"Accurracy para voted Classifier: {nltk.classify.accuracy(voted_classifier, testing_set)*100}")

NameError: ignored